In [1]:
import pandas as pd
from catboost import CatBoostClassifier, Pool
import numpy as np
import gc

In [3]:
df_all = pd.read_feather('/home/kai/talkingdata/data/ALL_features_supplementV3_feature42.ftr')
target = 'is_attributed'
extra = ['ip_app_device_os_channel_regression']
feature_cols = list(set(df_all.columns) - set([target]) -set(extra))

In [7]:
from sklearn.metrics import roc_auc_score

target = 'is_attributed'

params = {
    'eval_metric': 'AUC',
         'learning_rate':0.35, 
         'loss_function':'Logloss', 
         'depth':6, 
         'iterations':200,
         'scale_pos_weight': 99,
        'l2_leaf_reg': 9,
        'one_hot_max_size': 200,
    'leaf_estimation_method': 'Gradient',
    'rsm': 0.6,
#     'max_ctr_complexity':2,
        }


categorical_col = [ 'app', 'device', 'os', 'channel', 'hour']


category_index = [feature_cols.index(cat) for cat in categorical_col]

In [8]:
def train_catboost(x_train, x_val, feature_cols, category_index, params, best_round = None, target='is_attributed'):
    param = params.copy()    
    print('Start training')
    model = CatBoostClassifier(**param)
    model.fit(x_train[feature_cols],x_train[target],  
              cat_features=category_index, 
              use_best_model=True, 
              verbose_eval=1 )
    return model

## Train Catboost

In [ ]:
save_model_name = '/home/kai/talkingdata/data/all_suppelement_tree_200_depth6_scale99_lr0.35_ff0.6_onehot200'
model = train_catboost(df_all, None, feature_cols,category_index,params,None,target)
model.save_model(save_model_name)

Start training
You should provide test set for use best model. use_best_model parameter swiched to false value.
0:	learn: 0.9660803	total: 2m 33s	remaining: 8h 28m 55s
1:	learn: 0.9702093	total: 4m 59s	remaining: 8h 14m 30s
2:	learn: 0.9714373	total: 7m 34s	remaining: 8h 17m 17s
3:	learn: 0.9727371	total: 10m 12s	remaining: 8h 20m 26s
4:	learn: 0.9741021	total: 12m 58s	remaining: 8h 26m 8s
5:	learn: 0.9762021	total: 15m 39s	remaining: 8h 26m 20s
6:	learn: 0.9768180	total: 18m 17s	remaining: 8h 24m 25s
7:	learn: 0.9778866	total: 21m 9s	remaining: 8h 27m 49s
8:	learn: 0.9784334	total: 24m 8s	remaining: 8h 32m 17s
9:	learn: 0.9784131	total: 26m 55s	remaining: 8h 31m 31s
10:	learn: 0.9791329	total: 30m 3s	remaining: 8h 36m 19s
11:	learn: 0.9794107	total: 33m 23s	remaining: 8h 43m 12s
12:	learn: 0.9796444	total: 36m 31s	remaining: 8h 45m 22s
13:	learn: 0.9801743	total: 39m 24s	remaining: 8h 43m 35s
14:	learn: 0.9803902	total: 42m 6s	remaining: 8h 39m 19s
15:	learn: 0.9806397	total: 44m 57s	

In [ ]:
import json
feature_file = '/home/kai/talkingdata/data/catboost-featurecolsV3_col38_depth6_scale99_tree200_lr0.35_ff0.6_onehot200.json'

with open(feature_file, 'w') as outfile:
    json.dump(feature_cols, outfile)